In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from riotwatcher import LolWatcher, ApiError


In [4]:
lol_watcher = LolWatcher('RGAPI-a2e5141b-98d9-427a-905c-594767fb16cf')
my_region = 'euw1'


In [95]:
def liste_player_all_ranks():
    list_finale = []
    list_finale = pd.DataFrame(list_finale, columns=['puuid'])
    random_page = np.random.randint(1, 10)
    for tier in ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']:
        for division in ['I', 'II', 'III', 'IV']:
            try:
                listloc = lol_watcher.league.entries(my_region,
                                                   queue='RANKED_SOLO_5x5',
                                                   tier=tier,
                                                   division=division,
                                                   page=random_page)
                listloc = pd.DataFrame(listloc)
                listloc = pd.DataFrame(listloc['puuid'].drop_duplicates().reset_index(drop=True))
                listloc.columns = ['puuid']
                list_finale = pd.concat([list_finale, listloc], ignore_index=True)
                print(f"Loaded {len(listloc)} players from {tier} {division}")
            except ApiError as err:
                print(f"Error fetching data for {tier} {division}: {err}")
    
    try:
        listlocchall = lol_watcher.league.challenger_by_queue(my_region, 'RANKED_SOLO_5x5')
        listlocchall = pd.DataFrame(listlocchall)
        listchallpuuid = listlocchall['entries'].apply(lambda x: x['puuid'])
        listlocchall = pd.DataFrame(listchallpuuid).drop_duplicates().reset_index(drop=True)
        listlocchall.columns = ['puuid']
        listlocchall = pd.DataFrame(listlocchall['puuid'].sample(205).drop_duplicates().reset_index(drop=True))
        listlocchall.columns = ['puuid']
        list_finale = pd.concat([list_finale, listlocchall], ignore_index=True)
        print(f"Loaded {len(listlocchall)} players from challenger")
    except ApiError as err:
        print(f"Error fetching data for challenger: {err}")
    
    try:
        listlocmaster = lol_watcher.league.masters_by_queue(my_region, 'RANKED_SOLO_5x5')
        listlocmaster = pd.DataFrame(listlocmaster)
        listmasterpuuid = listlocmaster['entries'].apply(lambda x: x['puuid'])
        listlocmaster = pd.DataFrame(listmasterpuuid).drop_duplicates().reset_index(drop=True)
        listlocmaster.columns = ['puuid']
        listlocmaster = pd.DataFrame(listlocmaster['puuid'].sample(410).drop_duplicates().reset_index(drop=True))
        listlocmaster.columns = ['puuid']
        list_finale = pd.concat([list_finale, listlocmaster], ignore_index=True)
        print(f"Loaded {len(listlocmaster)} players from master")
    except ApiError as err:
        print(f"Error fetching data for master: {err}")

    try:
        listlocgrandmaster = lol_watcher.league.grandmaster_by_queue(my_region, 'RANKED_SOLO_5x5')
        listlocgrandmaster = pd.DataFrame(listlocgrandmaster)
        listgrandmasterpuuid = listlocgrandmaster['entries'].apply(lambda x: x['puuid'])
        listlocgrandmaster = pd.DataFrame(listgrandmasterpuuid).drop_duplicates().reset_index(drop=True)
        listlocgrandmaster.columns = ['puuid']
        listlocgrandmaster = pd.DataFrame(listlocgrandmaster['puuid'].sample(205).drop_duplicates().reset_index(drop=True))
        listlocgrandmaster.columns = ['puuid']
        list_finale = pd.concat([list_finale, listlocgrandmaster], ignore_index=True)
        print(f"Loaded {len(listlocgrandmaster)} players from grand master")
    except ApiError as err:
        print(f"Error fetching data for grand master: {err}")
    
    list_finale = list_finale.drop_duplicates().reset_index(drop=True)
    return list_finale


In [96]:
df_test_2 = liste_player_all_ranks()

Loaded 205 players from IRON I
Loaded 205 players from IRON II
Loaded 205 players from IRON III
Loaded 205 players from IRON IV
Loaded 205 players from BRONZE I
Loaded 205 players from BRONZE II
Loaded 205 players from BRONZE III
Loaded 205 players from BRONZE IV
Loaded 205 players from SILVER I
Loaded 205 players from SILVER II
Loaded 205 players from SILVER III
Loaded 205 players from SILVER IV
Loaded 205 players from GOLD I
Loaded 205 players from GOLD II
Loaded 205 players from GOLD III
Loaded 205 players from GOLD IV
Loaded 205 players from PLATINUM I
Loaded 205 players from PLATINUM II
Loaded 205 players from PLATINUM III
Loaded 205 players from PLATINUM IV
Loaded 205 players from DIAMOND I
Loaded 205 players from DIAMOND II
Loaded 205 players from DIAMOND III
Loaded 205 players from DIAMOND IV
Loaded 205 players from challenger
Loaded 410 players from master
Loaded 205 players from grand master


In [98]:
df_test_2.head()

,puuid
0,S9m6fETkHbJ52_AxPuOqEWfu-06vo4iAEg7FMxiO0CzWV_...
1,vvGyuoHpg9YQArvIHlPpXzDJHOTMfG8LEl83_TzX_1Aw-e...
2,6UADXjLomq-3y8LndKve2Z7FUS3ETtrIdyqbyOiLvPFMTa...
3,JLUEPk9OB-NrqNOQqF78oF434Blp04bNGb6tqU-aUnHJJ8...
4,D2xDwY9cAkNMVUylBa7NVup7qVCBfVvBaYY4Rw3tuCoI79...


In [60]:
def last_ranked_game_by_puuid(puuid):
    try:
        matchlist = lol_watcher.match.matchlist_by_puuid(my_region, puuid, queue='RANKED_SOLO_5x5', count=1)
        return matchlist  
    except ApiError as err:
        print(f"Error fetching match list for {puuid}: {err}")
    return None

In [65]:
game_test = last_ranked_game_by_puuid(df_test.iloc[-1]['puuid'])

KeyError: 'puuid'

In [ ]:
def create_last_game_df(puuid_df):
    last_games = []
    for puuid in puuid_df['puuid']:
        last_game_id = last_ranked_game_by_puuid(puuid)
        if last_game_id:
            last_games.append(last_game_id)
        else:
            last_games.append(None)
    last_game_df = pd.DataFrame(last_games)
    return puuid_df

In [34]:
def did_player_won_game(puuid, game_id):
    try:
        game = lol_watcher.match.by_id(my_region, game_id)
        for participant in game['info']['participants']:
            if participant['puuid'] == puuid:
                return participant['win']
    except ApiError as err:
        print(f"Error fetching game data: {err}")
    return False

In [57]:
puuidtest = list2['puuid'].iloc[0]
gametestli = lol_watcher.match.matchlist_by_puuid(my_region, puuidtest)
gametest = gametestli[0]

In [58]:
print(did_player_won_game(puuidtest, gametestli[2]))

True
